In [1]:
import sys
import json
params = {}
default = {'training_folder': '../../scenarios/houston/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston/', 'params': {}}


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

PyTables is not installed. No support for HDF output.
num cores: 8


In [3]:
import sys
sys.path.insert(0, r'../../../quetzal')
from quetzal.model import stepmodel
from quetzal.io import excel
import pandas as pd
if manual:
    %matplotlib inline

In [4]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')

In [5]:
sm = stepmodel.read_zippedpickles(model_folder +'los')
sm.segments = ['car_owner', 'pt_captive'] 

zones: 100%|██████████| 28/28 [00:04<00:00,  6.64it/s]           


In [6]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False)
kwargs = var['preparation_logit'].to_dict()

utility_scale = var['preparation_logit']['time']
kwargs['transfers'] = -kwargs['transfers']/utility_scale # From min/transfer to pt/transfer
kwargs['price'] = -60/kwargs['price']/utility_scale # From 
kwargs['time'] = -1/utility_scale/60 # From min/pt to pt/sec
kwargs


{'time': -0.0008333333333333334,
 'price': -0.2,
 'mode': 1,
 'pt_mode': 0.5,
 'pt_path': 0.1,
 'transfers': -0.25}

In [7]:
sm.preparation_logit(
    segments=['car_owner', 'pt_captive'],
    **kwargs
)

In [8]:
sm.utility_values.T

value,time,price,ntransfers,mode_utility
segment,,,,
root,-0.000833,-0.2,-0.25,1.0
car_owner,-0.000833,-0.2,-0.25,1.0
pt_captive,-0.000833,-0.2,-0.25,1.0


In [9]:
sm.logit_scales.T 

route_type,root,bus,walk,car,tram,pt
segment,,,,,,
root,1.0,0.1,0.0,0.0,0.1,0.5
car_owner,1.0,0.1,0.0,0.0,0.1,0.5
pt_captive,1.0,0.1,0.0,0.0,0.1,0.5


In [10]:
sm.mode_utility.T

route_type,root,bus,walk,car,tram
segment,,,,,
root,0,0,0,0,0
car_owner,0,0,0,0,0
pt_captive,0,0,0,0,0


In [11]:
sm.mode_utility.loc[ 'car', 'pt_captive'] = -100
sm.mode_utility.T

route_type,root,bus,walk,car,tram
segment,,,,,
root,0,0,0,0,0
car_owner,0,0,0,0,0
pt_captive,0,0,0,-100,0


In [12]:
sm.mode_nests.T 

route_type,root,bus,walk,car,tram,pt
segment,,,,,,
root,NaN,pt,root,root,pt,root
car_owner,NaN,pt,root,root,pt,root
pt_captive,NaN,pt,root,root,pt,root


In [13]:
sm.pt_los['ntransfers'] = sm.pt_los['alighting_links'].apply(lambda x: max(len(x)-1, 0))
sm.pt_los['price'] = 0  ######
sm.car_los['ntransfers'] = 0
sm.car_los['price'] = 0

In [14]:
sm.analysis_pt_time(
        boarding_time=None,
        alighting_time=None,
        walk_on_road=True,
    )
 

sm.analysis_car_route_type() 

sm.los = pd.concat([sm.pt_los, sm.car_los]).reset_index(drop=True)
sm.los['path'] = sm.los['path'].apply(lambda p: tuple(p)) 


sm.analysis_mode_utility()

100%|██████████| 2/2 [00:00<00:00, 34.40it/s]


In [15]:
sm.initialize_logit()
sm.step_logit()

100%|██████████| 10/10 [00:01<00:00,  8.66it/s]


In [16]:
sm.volumes.fillna(0, inplace=True)
sm.compute_los_volume(keep_segments=False)

In [17]:
sm.los['volume'].sum()

3574821.0000000005

In [18]:
sm.pt_los = sm.los.copy()
sm.pt_los = sm.pt_los.loc[sm.pt_los['route_type'] != 'car']

In [19]:
sm.step_pt_assignment(
    road=True,
    boardings=True,
    alightings=True,
    transfers=True
 
)

pt_captive: 100%|██████████| 2/2 [00:05<00:00,  2.71s/it]


In [20]:
sm.links.rename(columns={'load': 'volume'}, inplace=True)
sm.analysis_pt_length(
     walk_on_road=True
)
sm.analysis_car_length()
sm.analysis_pt_time(
     walk_on_road=True
)

In [21]:
sm.summary_link_max(inplace=True)
sm.summary_link_sum(inplace=True)
sm.summary_od(inplace=True)

In [22]:
sm.los = pd.concat([sm.pt_los, sm.car_los]).reset_index(drop=True)

In [23]:
sm.summary_path_average(inplace=True)
sm.summary_aggregated_path_average(inplace=True, pt_route_types=['subway', 'bus', 'tram', 'rail'])
sm.summary_path_sum(inplace=True)

s = sm.stack_link_sum.unstack([-1, -2]).groupby(level=0).sum().stack([0, 1])
s.name = 'sum'
sm.stack_aggregated_link_sum = s 
s = sm.stack_link_max.unstack().groupby(level=0).max().stack()
s.name = 'max'
sm.stack_aggregated_link_max = s

In [24]:
sm.od_los.loc['zone_106', 'zone_129']


segment     car_owner                                                        \
sum            volume price          time in_vehicle_time in_vehicle_length   
mode                                                                          
car      0.000000e+00   0.0  0.000000e+00    0.000000e+00      0.000000e+00   
tram     6.047316e-20   0.0  7.774276e-17    4.282039e-18      6.194908e-17   

segment               pt_captive                                      \
sum        ntransfers     volume price          time in_vehicle_time   
mode                                                                   
car      0.000000e+00   0.000000   0.0      0.000000             0.0   
tram     1.165815e-21  25.009342   0.0  66685.327528          3673.0   

segment                                    root                      \
sum     in_vehicle_length ntransfers     volume price          time   
mode                                                                  
car                   0.0        0.0   0.000000   0.0      0.000000   
tram              53138.0        1.0  25.009342   0.0  66685.327528   

segment                                               
sum     in_vehicle_time in_vehicle_length ntransfers  
mode                                                  
car                 0.0               0.0        0.0  
tram             3673.0           53138.0        1.0

In [25]:
print(sm.pt_los.set_index(['origin', 'destination']).loc['zone_106', 'zone_129'].iloc[0]['path'])

('zone_106', 'road_node_8079', 'road_node_25377', 'road_node_31024', 'road_node_49926', 'road_node_56485', 'road_node_1707', 'road_node_29172', 'road_node_51594', 'road_node_42203', 'road_node_62672', 'road_node_32997', 'road_node_3997', 'road_node_49132', 'road_node_2737', 'road_node_19881', 'road_node_70504', 'road_node_10981', 'road_node_47146', 'road_node_80287', 'road_node_39933', 'road_node_45626', 'road_node_7001', 'road_node_49703', 'road_node_20168', 'road_node_8195', 'road_node_84231', 'road_node_56186', 'road_node_16294', 'road_node_78505', 'road_node_25287', 'road_node_34212', 'road_node_38843', 'road_node_31283', 'road_node_41353', 'road_node_31014', 'road_node_5493', 'road_node_36502', 'road_node_64135', 'road_node_4704', 'road_node_1302', 'road_node_66186', 'road_node_26037', 'road_node_67768', 'road_node_74318', 'road_node_84812', 'road_node_31145', 'road_node_30919', 'road_node_68216', 'road_node_66449', 'road_node_21655', 'road_node_20699', 'road_node_63416', 'road_no

In [26]:
sm.probabilities.groupby(['segment'])['pt'].describe()

,count,mean,std,min,25%,50%,75%,max
segment,,,,,,,,
car_owner,67776.0,0.005006,0.021352,0.0,1.826857e-12,4.690775e-07,0.000312,0.400227
pt_captive,67776.0,0.917440,0.273396,0.0,1.000000e+00,1.000000e+00,1.000000,1.000000


In [27]:
sm.probabilities.loc[sm.probabilities['segment'] == 'car_owner'].sort_values(by='pt', ascending=False).head()

route_type,origin,destination,segment,root,car,walk,pt,tram,bus
9882,zone_58,zone_242,car_owner,1,0.585051,0.014722,0.400227,0.0,0.400227
4798,zone_274,zone_196,car_owner,1,0.609733,0.013303,0.376964,0.0,0.376964
11544,zone_445,zone_109,car_owner,1,0.629095,0.000000,0.370905,0.0,0.370905
2354,zone_196,zone_274,car_owner,1,0.619537,0.013541,0.366922,0.0,0.366922
5804,zone_242,zone_58,car_owner,1,0.624947,0.014591,0.360462,0.0,0.360462


In [28]:
sm.links['boardings'].sum()


1801937.8909483752

In [29]:
sm.car_los

,origin,destination,time,path,gtime,node_path,link_path,ntlegs,ntransfers,price,route_types,route_type,access_length,in_vehicle_length,length
0,zone_214,zone_466,8654.650617,"[zone_214, road_node_26423, road_node_85525, r...",8654.650617,"[road_node_26423, road_node_85525, road_node_4...","[road_link_21698, road_link_21788, road_link_5...","[(zone_214, road_node_26423), (road_node_51078...",0,0,"(car,)",car,3066.472274,121630.748735,124697.221008
1,zone_398,zone_109,2788.090019,"[zone_398, road_node_29487, road_node_73513, r...",2788.090019,"[road_node_29487, road_node_73513, road_node_6...","[road_link_67045, road_link_56380, road_link_3...","[(zone_398, road_node_29487), (road_node_33974...",0,0,"(car,)",car,1160.507942,35850.438139,37010.946080
2,zone_266,zone_367,3440.078310,"[zone_266, road_node_4622, road_node_21406, ro...",3440.078310,"[road_node_4622, road_node_21406, road_node_86...","[road_link_8606_r, road_link_2791, road_link_2...","[(zone_266, road_node_4622), (road_node_63066,...",0,0,"(car,)",car,1131.337761,48150.297710,49281.635471
3,zone_139,zone_334,31419.875096,"[zone_139, road_node_2797, road_node_18377, ro...",31419.875096,"[road_node_2797, road_node_18377, road_node_69...","[road_link_49351_r, road_link_57196_r, road_li...","[(zone_139, road_node_2797), (road_node_40373,...",0,0,"(car,)",car,21751.207239,82887.199896,104638.407135
4,zone_176,zone_298,2886.719833,"[zone_176, road_node_2114, road_node_62594, ro...",2886.719833,"[road_node_2114, road_node_62594, road_node_73...","[road_link_58616, road_link_25944, road_link_3...","[(zone_176, road_node_2114), (road_node_38584,...",0,0,"(car,)",car,1177.393613,35455.494131,36632.887744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67771,zone_386,zone_257,1688.915957,"[zone_386, road_node_7239, road_node_25861, ro...",1688.915957,"[road_node_7239, road_node_25861, road_node_33...","[road_link_74780, road_link_73668, road_link_7...","[(zone_386, road_node_7239), (road_node_66565,...",0,0,"(car,)",car,662.366825,20227.889345,20890.256169
67772,zone_41,zone_454,7281.064211,"[zone_41, road_node_86038, road_node_62171, ro...",7281.064211,"[road_node_86038, road_node_62171, road_node_5...","[road_link_22174, road_link_49844, road_link_5...","[(zone_41, road_node_86038), (road_node_9797, ...",0,0,"(car,)",car,2678.262539,101720.653675,104398.916214
67773,zone_328,zone_154,3079.289932,"[zone_328, road_node_89141, road_node_2405, ro...",3079.289932,"[road_node_89141, road_node_2405, road_node_30...","[road_link_69984, road_link_70062, road_link_3...","[(zone_328, road_node_89141), (road_node_58137...",0,0,"(car,)",car,360.938082,67134.240103,67495.178185
67774,zone_170,zone_169,2112.345012,"[zone_170, road_node_81193, road_node_20522, r...",2112.345012,"[road_node_81193, road_node_20522, road_node_2...","[road_link_75273, road_link_38907, road_link_1...","[(zone_170, road_node_81193), (road_node_82465...",0,0,"(car,)",car,493.536800,35366.326010,35859.862811


In [30]:
sm.utilities

route_type,origin,destination,segment,root,car,walk,pt,tram,bus
0,zone_117,zone_1,car_owner,-2.577398,-2.577428,-inf,-12.976466,-inf,-12.976466
1,zone_117,zone_1,pt_captive,-12.976466,-102.577428,-inf,-12.976466,-inf,-12.976466
2,zone_117,zone_103,car_owner,-2.811170,-2.811171,-inf,-16.367248,-inf,-16.367248
3,zone_117,zone_103,pt_captive,-16.367248,-102.811171,-inf,-16.367248,-inf,-16.367248
4,zone_117,zone_104,car_owner,-1.491705,-1.492626,-inf,-8.482256,-inf,-8.482256
...,...,...,...,...,...,...,...,...,...
12387,zone_91,zone_77,pt_captive,-59.284387,-116.443876,-inf,-59.284387,-inf,-59.284387
12388,zone_91,zone_82,car_owner,-19.764354,-19.764354,-inf,-inf,-inf,-inf
12389,zone_91,zone_82,pt_captive,-119.764354,-119.764354,-inf,-inf,-inf,-inf
12390,zone_91,zone_87,car_owner,-16.454675,-16.454675,-inf,-60.908422,-inf,-60.908422


In [31]:
sm.probabilities

route_type,origin,destination,segment,root,car,walk,pt,tram,bus
0,zone_117,zone_1,car_owner,1,9.999695e-01,0.0,3.046086e-05,0.0,3.046086e-05
1,zone_117,zone_1,pt_captive,1,1.221227e-39,0.0,1.000000e+00,0.0,1.000000e+00
2,zone_117,zone_103,car_owner,1,9.999987e-01,0.0,1.296194e-06,0.0,1.296194e-06
3,zone_117,zone_103,pt_captive,1,2.869996e-38,0.0,1.000000e+00,0.0,1.000000e+00
4,zone_117,zone_104,car_owner,1,9.990795e-01,0.0,9.205398e-04,0.0,9.205398e-04
...,...,...,...,...,...,...,...,...,...
12387,zone_91,zone_77,pt_captive,1,1.499511e-25,0.0,1.000000e+00,0.0,1.000000e+00
12388,zone_91,zone_82,car_owner,1,1.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00
12389,zone_91,zone_82,pt_captive,1,1.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00
12390,zone_91,zone_87,car_owner,1,1.000000e+00,0.0,4.942914e-20,0.0,4.942914e-20


In [33]:
sm.od_los

segment                  car_owner                                   \
sum                         volume price       time in_vehicle_time   
origin  destination mode                                              
zone_0  zone_1      bus   0.008931   0.0   4.285161        1.885705   
                    car   0.000000   0.0   0.000000        0.000000   
        zone_104    bus   0.684262   0.0  46.308404       32.382759   
                    car   0.000000   0.0   0.000000        0.000000   
        zone_109    bus   0.367206   0.0  56.616397       30.689838   
...                            ...   ...        ...             ...   
zone_99 zone_85     car   0.000000   0.0   0.000000        0.000000   
        zone_87     bus   0.000002   0.0   0.001500        0.000256   
                    car   0.000000   0.0   0.000000        0.000000   
        zone_90     car   0.000000   0.0   0.000000        0.000000   
                    walk  0.000001   0.0   0.000584        0.000000   

segment                                                  pt_captive        \
sum                      in_vehicle_length    ntransfers     volume price   
origin  destination mode                                                    
zone_0  zone_1      bus          29.907679  1.064368e-03   0.000000   0.0   
                    car           0.000000  0.000000e+00   0.000000   0.0   
        zone_104    bus         208.391380  1.359192e-02  50.702857   0.0   
                    car           0.000000  0.000000e+00   0.000000   0.0   
        zone_109    bus         434.771484  2.194744e-02   0.000000   0.0   
...                                    ...           ...        ...   ...   
zone_99 zone_85     car           0.000000  0.000000e+00   0.000000   0.0   
        zone_87     bus           0.003252  6.439803e-08  25.475000   0.0   
                    car           0.000000  0.000000e+00   0.000000   0.0   
        zone_90     car           0.000000  0.000000e+00   0.000000   0.0   
                    walk          0.000000  0.000000e+00   0.000000   0.0   

segment                                                                   \
sum                               time in_vehicle_time in_vehicle_length   
origin  destination mode                                                   
zone_0  zone_1      bus   12078.042640          5315.0           84297.0   
                    car       0.000000             0.0               0.0   
        zone_104    bus    6814.105732          4765.0           30664.0   
                    car       0.000000             0.0               0.0   
        zone_109    bus    7738.906510          4195.0           59429.0   
...                                ...             ...               ...   
zone_99 zone_85     car       0.000000             0.0               0.0   
        zone_87     bus   23299.386085          3970.0           50504.0   
                    car       0.000000             0.0               0.0   
        zone_90     car       0.000000             0.0               0.0   
                    walk  24321.227058             0.0               0.0   

segment                                   root                      \
sum                      ntransfers     volume price          time   
origin  destination mode                                             
zone_0  zone_1      bus         3.0   0.008931   0.0      4.285161   
                    car         0.0   0.000000   0.0      0.000000   
        zone_104    bus         2.0  51.387119   0.0   6723.986938   
                    car         0.0   0.000000   0.0      0.000000   
        zone_109    bus         3.0   0.367206   0.0     56.616397   
...                             ...        ...   ...           ...   
zone_99 zone_85     car         0.0   0.000000   0.0      0.000000   
        zone_87     bus         1.0  25.475002   0.0  23299.384609   
                    car         0.0   0.000000   0.0      0.000000   
        zone_90     c

In [32]:
sm.to_zippedpickles(model_folder + 'logit_assignment')

stack_aggregated_link_max: 100%|██████████| 47/47 [00:25<00:00,  1.85it/s]
